In [87]:
import pandas as pd
from pandas import read_csv
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
def read_csv(folder):
    #needs args
    train_df = pd.read_csv(folder + 'split/train.csv')
    val_df = pd.read_csv(folder + 'split/validation.csv')
    test_df = pd.read_csv(folder + 'split/test.csv')
    return train_df, val_df, test_df 

In [77]:
def tokenize(data_frame):
    counter = 0
    sent1_tokenized = []
    tenp = len(data_frame) / 5
    ten = 0
    labels = []
    print("Fetching Labels")
    for i in data_frame['label']:
        labels.append(i)
    
    print("Tokenizing sentence 1")
    for i in data_frame['sent1']:
        if counter % tenp == 0:
            print(str(ten) + '% tokenized')
            ten += 10
        sent1_tokenized.append([re.sub('-',' ',x.lower()) for x in word_tokenize(i)])
        counter +=1 

    sent2_tokenized = []
    counter = 0
    print("Tokenizing sentence 2")
    for i in data_frame['sent2']:
        if counter % tenp == 0:
            print(str(ten) + '% tokenized')
            ten += 10
        sent2_tokenized.append([re.sub('-',' ',x.lower()) for x in word_tokenize(i)])
        counter +=1 

    print("Tokens Generated")
    
    
    return sent1_tokenized, sent2_tokenized, labels

In [88]:
def remove_stopwords(tokenized_sent):
    stop_words = stopwords.words('english')
    stop_words += string.punctuation
    for i in tokenized_sent:
        if i in stop_words or type(i) == int:
            tokenized_sent.remove(i)

    return tokenized_sent
    

In [28]:
def post_processing(sent1_tokenized, sent2_tokenized):
    counter = 0
    ten = 0
    tenper = len(sent1_tokenized) / 5

    print('Post Processing sentence 1')
    for i in sent1_tokenized:
        counter += 1
        if counter % tenper == 0:
            print(str(ten) + '% processed')
            ten += 10
        i = remove_stopwords(i)

    print('Post Processing sentence 2')
    counter = 0
    for i in sent2_tokenized:
        counter += 1
        if counter % tenper == 0:
            print(str(ten) + '% processed')
            ten += 10
        i = remove_stopwords(i)
        
    print('Finished')
    
    return sent1_tokenized, sent2_tokenized

In [6]:
def gen_vocab(sent1_tokenz, sent2_tokenz):
    raw_text = []
    for i in sent1_tokenz:
        raw_text += i
    for i in sent2_tokenz:
        raw_text += i
    
    int2char = dict(enumerate(set(raw_text)))
    vocab = {char : num for num, char in int2char.items()}
    return vocab
        

In [7]:
train_df, val_df, test_df = read_csv('data/')

In [78]:
train_sent1_tokenized, train_sent2_tokenized, labels = tokenize(train_df)

Fetching Labels
Tokenizing sentence 1
0% tokenized
10% tokenized
20% tokenized
30% tokenized
40% tokenized
Tokenizing sentence 2
50% tokenized
60% tokenized
70% tokenized
80% tokenized
90% tokenized
Tokens Generated


In [86]:
string.punctuation


AttributeError: 'str' object has no attribute 'punctuation'

In [89]:
proc_sent1, proc_sent2 = post_processing(train_sent1_tokenized, train_sent2_tokenized)

Post Processing sentence 1
0% processed
10% processed
20% processed
30% processed
40% processed
Post Processing sentence 2
50% processed
60% processed
70% processed
80% processed
90% processed
Finished


In [96]:
target_vocab = gen_vocab(train_sent1_tokenized, train_sent2_tokenized)

In [ ]:
a = [[1],[2,3],[3,2,1]]

In [ ]:
b = [[3,4],[3,4,5],[6,6,6]]

In [ ]:
c = [[0],[0],[1]]

In [ ]:
data = list(zip(a,b,c))

In [ ]:
df = pd.DataFrame(data, columns=['a','b','c'])

In [ ]:
df

In [92]:
def create_df(sent1,sent2,labels):
    df = pd.DataFrame(data=list(zip(sent1,sent2,labels)), columns=['sent1','sent2','labels'])
    df.to_csv('data/processed_training.csv')
    return df

In [93]:
processed_train_df = create_df(proc_sent1, proc_sent2, labels)

In [48]:
def zipping_data(df):
    X_train = []
    y_train = []
    for idx, row in df.iterrows():
        sent_1 = row['sent1']
        sent_2 = row['sent2']
        X_train.append((sent_1,sent_2))
        y_train.append(row['labels'])
        
    return X_train, y_train
       
        

In [49]:
X_train, Y_train = zipping_data(processed_train_df)

In [50]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class DebatesSet(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
        
    def __len__(self):
        return len(self.x)

In [ ]:
train_data = DebatesSet(X_train, Y_train)

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=30, shuffle=True)

In [54]:
import bcolz, pickle

In [55]:
glove_path = 'vectors/gLove/'

In [56]:
vectors = bcolz.open(f'{glove_path}/6B.50.dat')[:]

In [57]:
words = pickle.load(open(f'{glove_path}/6B.50_words.pkl', 'rb'))

In [58]:
word2idx = pickle.load(open(f'{glove_path}/6B.50_idx.pkl', 'rb'))

In [59]:
glove = {w: vectors[word2idx[w]] for w in words}


In [60]:
import numpy as np

In [61]:
matrix_len = len(vocab)

In [100]:
import torch

In [103]:
weights_matrix = np.zeros((matrix_len, 50))

In [104]:
words_found = 0

In [113]:
emb_dim = np.array(torch.Tensor(weights_matrix).size())

In [114]:
np.array(emb_dim)

array([63819,    50])

In [124]:
matrix_len = len(target_vocab)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

glove_vocab = {}

for i, word in enumerate(target_vocab):
    try: 
        glove_vocab[word] = glove[word]
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(50,))


In [70]:
import re


In [131]:
from torch import nn

In [132]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings = len(weights_matrix)
    embedding_dim = 50
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [138]:
emb_layer, num_embeddings, embedding_dim = create_emb_layer(torch.Tensor(weights_matrix))

In [143]:

import pickle

In [145]:

f = open('pretrain_vectors/glove/glove_vocab.pkl','wb')
pickle.dump(glove_vocab,f)

In [146]:
f.close()